In [1]:
rm(list=ls())
suppressPackageStartupMessages(require(skimr))
suppressPackageStartupMessages(require(readxl))
suppressPackageStartupMessages(require(stringr))
suppressPackageStartupMessages(require(dplyr))
suppressPackageStartupMessages(require(ggplot2))
suppressPackageStartupMessages(require(forecast))

In [2]:
#------------------------------------------------------
# data pre treatment (similar to univariate case)

df <- read_excel("data/data.xlsx",sheet = 3)
tmp <-df%>% filter(aire=="44_GRAND EST" & champ=="ESS")

df <-tmp[-which(!is.na(tmp$secret_stat)),]%>%select(-secret_stat)
df <- df %>% select(-champ,-type_aire,-type_naf,-aire)
df <- df[,c(TRUE,str_detect(colnames(df),"nb_etab")[-1])]
tmp <- t(df)
colnames(tmp) <- tmp[1,]
tmp <-tmp[-1,]
df <- tmp
tmp <-df%>%as.data.frame
tmp <-cbind("date"=rep(NA,nrow(tmp)),tmp)
for (i in 1:nrow(tmp)){
    tmp.format.date <- strsplit(rownames(df)[i],"nb_etab")[[1]][2]
    subsplit <-strsplit(tmp.format.date,"T")[[1]]
    year<-subsplit[1]
    trimester <- subsplit[2]
    month <-switch(trimester,
        "1"="01",
        "2"="04",
        "3"="07",
        "4"="10"
    )
    tmp[i,1] <- paste(year,month,"01",sep="-")
}
tmp$date <-as.Date(tmp$date)
rownames(tmp) <- NULL

df.indexes <-which(tmp$date < as.Date("2020-01-01"))
train.indexes<- which(tmp$date < as.Date("2018-01-01") )
test.indexes <-which(tmp$date>= as.Date("2018-01-01") & tmp$date <as.Date("2020-01-01") )
tmp <-tmp %>%select(-date)%>% mutate_all(as.numeric)
#----------------------------
# Eliminate total serie
tmp <- tmp[,-ncol(tmp)]
df.ts <- ts(tmp[df.indexes,],start=c(2010,1),frequency=4)# S'arrete à 2019 Q4
train.ts <-ts(tmp[train.indexes,],start=c(2010,1),frequency = 4)# S'arrete à 2017 Q4
test.ts <-ts(tmp[test.indexes,],start = c(2018,1),frequency = 4)# Commence a 2018 Q1 --> 2019 Q4
global.ts <- ts(tmp,start=c(2010,1),frequency=4)# Commence a 2010 Q1 --> 2021 Q2
covid.ts <- ts(tmp[(df.indexes[length(df.indexes)]+1):nrow(tmp),],start=c(2020,1),frequency = 4)# Commence a 2020 Q1 --> 2021 Q2

In [3]:
require(tensorflow)
require(keras)

Loading required package: tensorflow

Loading required package: keras



In [4]:
lab.to.num <-cbind("lab"=colnames(df.ts),"num"=1:length(colnames(df.ts))) %>%as.data.frame
lab.to.num$num <- as.numeric(lab.to.num$num)
colnames(df.ts) <-lab.to.num$num

In [5]:

df.train <-df.ts[1:(dim(df.ts)[1]*3/4),] %>%as.data.frame
df.test <-df.ts[((dim(df.ts)[1]*3/4)+1):dim(df.ts)[1],] %>%as.data.frame
df.train<-df.train[,1]
df.test<-df.test[,1]


In [6]:
#-----------------------------
# Scaled the train sample and get the sample factor parmas: mean + sd
scaled_train <- scale(df.train)
scale_factor <-c(attr(scaled_train,"scaled:center"),attr(scaled_train,"scaled:scale"))


In [7]:
#df <- df.ts %>% as.data.frame()
#df.scaled <-scale(df)
#centered_scale <-attr(df.scaled,"scaled:center")
#sd_scale <-attr(df.scaled,"scaled:scale")
#write.csv(df,"data\\multi_TS.csv")

In [8]:
#----------------
# We d like to do a 6 Quarter forecast,
# We need to base each prediction on 6 points
prediction <- 6
lag <- prediction

In [9]:
#scaled_train <- as.matrix(scaled_train)
#scaled_train
#-------------------
#We lag the data 5 times
x_train_data <-t(sapply(1:(length(scaled_train)-lag- prediction+1),
    function(x){scaled_train[x:(x+lag-1),1]}
))

#Transform into array
x_train_array <- array(
    data=as.numeric(unlist(x_train_data)),
    dim=c(nrow(x_train_data),
    lag,
    1)
)
# An array with the third dimension fixed to one , and with the other plan forming the matrix of size : 19 . 6


In [10]:
scale_factor

[1] 1233.90000   43.68409

In [11]:
x_train_data

1.28422032,1.14687055,1.03241241,1.00952078,0.96373753,0.91795427
1.14687055,1.03241241,1.00952078,0.96373753,0.91795427,0.80349613
1.03241241,1.00952078,0.96373753,0.91795427,0.80349613,0.75771288
1.00952078,0.96373753,0.91795427,0.80349613,0.75771288,0.57457986
0.96373753,0.91795427,0.80349613,0.75771288,0.57457986,0.52879660
0.91795427,0.80349613,0.75771288,0.57457986,0.52879660,0.50590497
0.80349613,0.75771288,0.57457986,0.52879660,0.50590497,0.50590497
0.75771288,0.57457986,0.52879660,0.50590497,0.50590497,0.29988032
0.57457986,0.52879660,0.50590497,0.50590497,0.29988032,0.23120544
0.52879660,0.50590497,0.50590497,0.29988032,0.23120544,0.13963893
0.50590497,0.50590497,0.29988032,0.23120544,0.13963893,-0.02060246


In [12]:
y_train_data <-  t(sapply(
    (1+lag):(length(scaled_train)-prediction +1),
    function(x){scaled_train[x:(x+prediction-1)]}
))
y_train_arr <-array(
    data=as.numeric(unlist(y_train_data)),
    dim=c(
        nrow(y_train_data),
        prediction,
        1
    )
)

In [13]:
x_test <- df.train[(nrow(scaled_train) -prediction +1):nrow(scaled_train)]
x_test_scaled <-(x_test -scale_factor[1])/scale_factor[2]
cat("X test :",x_test,"\nX test scaled :",x_test_scaled)

X test : 1191 1178 1174 1166 1160 1151 
X test scaled : -0.9820508 -1.279642 -1.371209 -1.554342 -1.691691 -1.897716

In [61]:
x_pred_arr <-array(
    data=x_test_scaled,
    dim=c(1,lag,1)
)

In [62]:
LSTM_model <- keras_model_sequential()

LSTM_model%>% layer_lstm(
    units=50,
    batch_input_shape=c(1,6,1),#BATCH size , timestep ,features
    return_sequences=T,
    stateful=T

)%>%
layer_dropout(rate=0.5)%>%layer_lstm(
    units =50,
    return_sequences=T,stateful=T
)%>% layer_dropout(rate=0.5)%>% time_distributed(keras::layer_dense(units=1))

Loaded Tensorflow version 2.9.1



In [63]:
LSTM_model %>% compile(loss='mae',optimizer='adam',metrics='accuracy')
summary(LSTM_model)

Model: "sequential"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 lstm_1 (LSTM)                      (1, 6, 50)                      10400       
 dropout_1 (Dropout)                (1, 6, 50)                      0           
 lstm (LSTM)                        (1, 6, 50)                      20200       
 dropout (Dropout)                  (1, 6, 50)                      0           
 time_distributed (TimeDistributed)  (1, 6, 1)                      51          
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
________________________________________________________________________________


In [68]:
LSTM_model %>% keras::fit(
    x=x_train_array,
    y=y_train_arr,
    batch_size=1,
    epochs=20,# Number of times to iterate over the training data
    verbose=2,
    shuffle=FALSE


)

In [78]:
LSTM_forecast <- LSTM_model %>% predict(x_pred_arr,batch_size=1) %>% .[,,1]
LSTM_forecast

[1] -1.139914 -1.104402 -1.119791 -1.105877 -1.107732 -1.098921

In [79]:
#-----------------------
# Rescale data to restore original values
LSTM_forecast <-LSTM_forecast* scale_factor[2]+ scale_factor[1]


In [83]:
fitted_values <-predict(LSTM_model,x_train_array,batch_size=1)%>% .[,,1]
fitted_values

-1.01684940,-0.79716766,-0.46121526,-0.052262656,0.26892287,0.42577264
0.49101019,0.49558792,0.48377255,0.466804773,0.44647920,0.41564864
0.40864208,0.40361437,0.39282906,0.375227422,0.34408143,0.30775937
0.30460393,0.30393976,0.29761434,0.275500029,0.24618626,0.19453846
0.19627266,0.20682013,0.20296925,0.188948363,0.14771682,0.09775375
0.10837404,0.12004001,0.12433695,0.097800426,0.05870419,0.01711526
0.02606983,0.04384083,0.03349189,0.008622117,-0.02160325,-0.04998516
-0.03339406,-0.03353662,-0.04595468,-0.064627863,-0.08332925,-0.13413289
-0.14427871,-0.14738554,-0.15087238,-0.153303951,-0.19077653,-0.24636799
-0.25414085,-0.24852559,-0.23725197,-0.260707796,-0.30458111,-0.36512765
-0.36907154,-0.35134622,-0.36160389,-0.390968561,-0.43848467,-0.50977039


In [87]:
fit <-c(fitted_values[,1],fitted_values[dim(fitted_values)[1],2:dim(fitted_values)[2]])
fitted <- fit *scale_factor[2]+ scale_factor[1]

In [ ]:
#------------------------------
Due to the fact that out forecast start with 6 values offset, we need ti provide artificial 